In [1]:
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from tqdm.notebook import tqdm
import pandas as pd
import torch
import torch.nn as nn
import random
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F



def Pose_Estimation(img_path):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    CFG = {
        'EPOCHS':1,
        'LEARNING_RATE':3e-8,
        # 'LEARNING_RATE':10,
        'BATCH_SIZE':1,
        'SEED':41
        }
    base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')

    options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        num_poses = 22,
        output_segmentation_masks=False)

    detector = vision.PoseLandmarker.create_from_options(options)
    
    save_x = []
    save_y = []
    save_z = []
    save_presence = []
    
        
    img = mp.Image.create_from_file(img_path)
    pose_landmarks_list = detector.detect(img).pose_landmarks
        
    if not pose_landmarks_list:
        return "전신 사진이어야 합니다!"
            
    save_x.append([i.x for i in pose_landmarks_list[0][11:33]])
    save_y.append([i.y for i in pose_landmarks_list[0][11:33]])
    save_z.append([i.z for i in pose_landmarks_list[0][11:33]])
    save_presence.append([i.presence for i in pose_landmarks_list[0][11:33]])
    
    df = pd.DataFrame(columns=['img_path','label'])
    
    df['img_path'] = None
    df['label'] = 0
    df['landmark_x'] = save_x
    df['landmark_y'] = save_y
    df['landmark_z'] = save_z
    df['landmark_presence'] = save_presence
    
    class CustomDataset(Dataset):
        def __init__(self, img_path ,landmark_x, landmark_y,	landmark_z,	landmark_presence, label):
            self.img_path = img_path
            self.landmark_x = landmark_x
            self.landmark_y = landmark_y
            self.landmark_z= landmark_z
            self.landmark_presence = landmark_presence
            self.label = label

        def __getitem__(self,index):

            result = np.concatenate((self.landmark_x[index] , self.landmark_y[index] , self.landmark_z[index] , self.landmark_presence[index]), axis=0)
            return result, self.label[index]

        def __len__(self):
            return len(self.landmark_x )
    
    val_dataset = CustomDataset(df["img_path"].values, df["landmark_x"].values, df["landmark_y"].values, df["landmark_z"].values,df["landmark_presence"].values, df["label"].values)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    class BaseModel(nn.Module):
        def __init__(self):
            super(BaseModel, self).__init__()
            self.classifier1 = nn.Linear(88, 20)
            # self.classifier1 = nn.Linear(22, 2)
            self.ReLU = nn.ReLU(inplace=True)
            self.classifier2 = nn.Linear(20, 2)


        def forward(self, x):
            x = self.classifier1(x)
            x = self.ReLU(x)
            x = self.classifier2(x)

            return F.log_softmax(x, dim=1)
        
    def validation(model, criterion, val_loader, device):
        model.eval()
        preds= []
        with torch.no_grad():
            for landmark_list, labels in tqdm(iter(val_loader)):
                landmark_list = landmark_list.float().to(device)
                pred = model(landmark_list)
                preds += pred.detach().argmax(1).cpu().numpy().tolist()
        return preds


    def train(model, optimizer, train_loader, val_loader, scheduler, device):
        model = model.to(device)
        criterion = nn.NLLLoss(weight=torch.tensor([0.01, 0.99]), reduction="sum").to(device)
        best_model = None
        for epoch in range(0, CFG['EPOCHS']):
            model.train()
            label = validation(model, criterion, val_loader, device)
        return best_model , label
    
    model = BaseModel()
    model.load_state_dict(torch.load('./Pose_Estimate_new.pt'))
    model.eval()
    
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
    infer_model, label = train(model, optimizer, None, val_loader, scheduler, device)
    
    if label[0] == 0:
        return "올바른 이미지 입니다"
    else:
        return "잘못된 이미지 입니다."

print(Pose_Estimation('person2.png'))
    




INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


  0%|          | 0/1 [00:00<?, ?it/s]

올바른 이미지 입니다
